In [1]:
import tensorflow as tf


device_name = tf.test.gpu_device_name()


if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
# import datetime
# import json
# import os
# import pprint
# import random
# import string
# import sys
# import tensorflow as tf

# assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
# TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# print('TPU address is', TPU_ADDRESS)

# from google.colab import auth
# auth.authenticate_user()
# with tf.compat.v1.Session(TPU_ADDRESS) as session:
#   print('TPU devices:')
#   pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  # with open('/content/adc.json', 'r') as f:
  #   auth_info = json.load(f)
  # tf.compat.v1.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

In [0]:
# import torch_xla_py.xla_model as xm

# device = xm.xla_device()

# xm.optimizer_step(optimizer)
# xm.mark_step()

In [4]:
import torch


if torch.cuda.is_available():    

   
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%matplotlib inline
import sys
import itertools
import numpy as np
import random as rn
import matplotlib.pyplot as plt
import torch
from torch import nn
import pandas as pd
import numpy as np

In [0]:
df_train = pd.read_csv('/content/drive/My Drive/New Colab Notebooks/task2_train_en.txt',sep = "\t",encoding = "ISO-8859-1")
df_test = pd.read_csv('/content/drive/My Drive/New Colab Notebooks/task2_test_en.txt',sep = "\t",encoding = "ISO-8859-1")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = df_train["tweet"],df_test["tweet"],df_train["class"],df_test["class"]

In [8]:
df_train.head()

,tweet,class
0,depression hurts cymbalta can help,0
1,jessicama right but cipro can make things much...,0
2,fibby are you on paxil i need help,0
3,redicine the lamotrigine and sjs just made cha...,0
4,have decided to skip my humira shot today my b...,0


In [9]:
!pip install transformers

     |████████████████████████████████| 573kB 14.3MB/s 
     |████████████████████████████████| 1.0MB 53.7MB/s 
     |████████████████████████████████| 890kB 56.9MB/s 
     |████████████████████████████████| 3.7MB 19.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=12d39e9e72e7a7983b5f3115840b56749e8cd2d61f8d9d8d5040389edd7c8b38
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 13.5MB/s 


In [0]:
# import os
# from pytorch_pretrained_bert.convert_tf_checkpoint_to_pytorch import convert_tf_checkpoint_to_pytorch
# path_bin = '/content/biobert_large/pytorch_model.bin'
# path_bert = '/content/biobert_large/'

# if (not os.path.exists(path_bin)):
#   convert_tf_checkpoint_to_pytorch(
#   path_bert + "bio_bert_large_1000k.ckpt",
#   path_bert + "config.json",
#   path_bert + "pytorch_model.bin"
#   )

# tokenizer = BertTokenizer.from_pretrained('/content/biobert_large', do_lower_case=False)

In [13]:
from transformers import BertTokenizer, RobertaForSequenceClassification, RobertaTokenizer
from transformers import AutoTokenizer, AutoModel
# tokenizer = BertTokenizer.from_pretrained("bert-large-")


print('Loading BERT tokenizer...')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large', do_lower_case=True)

Loading BERT tokenizer...


In [14]:

input_ids = []
attention_masks = []

.
for sent in X_train:

    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True, 
                        max_length = 64,          
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
    
       
    input_ids.append(encoded_dict['input_ids'])
    
    
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)


print('Original: ', X_train[0])
print('Token IDs:', input_ids[0])

Original:  depression hurts cymbalta can help
Token IDs: tensor([    0,  6943, 15774, 19258,  6648,  3967,   102,    64,   244,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1])


In [15]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)


train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size


train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

18,489 training samples
2,055 validation samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


batch_size = 32
 
train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )


validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [17]:
!conda env list

/bin/bash: conda: command not found


In [18]:
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = RobertaForSequenceClassification.from_pretrained(
    "roberta-large", 
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False, 
)


model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm

In [19]:

params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 395 different named parameters.

==== Embedding Layer ====

roberta.embeddings.word_embeddings.weight               (50265, 1024)
roberta.embeddings.position_embeddings.weight            (514, 1024)
roberta.embeddings.token_type_embeddings.weight            (1, 1024)
roberta.embeddings.LayerNorm.weight                          (1024,)
roberta.embeddings.LayerNorm.bias                            (1024,)

==== First Transformer ====

roberta.encoder.layer.0.attention.self.query.weight     (1024, 1024)
roberta.encoder.layer.0.attention.self.query.bias            (1024,)
roberta.encoder.layer.0.attention.self.key.weight       (1024, 1024)
roberta.encoder.layer.0.attention.self.key.bias              (1024,)
roberta.encoder.layer.0.attention.self.value.weight     (1024, 1024)
roberta.encoder.layer.0.attention.self.value.bias            (1024,)
roberta.encoder.layer.0.attention.output.dense.weight   (1024, 1024)
roberta.encoder.layer.0.attention.output.dense.bias          (

In [0]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters,
                  lr = 2e-5, 
                  eps = 1e-8 
                )

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs = 3

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [0]:
import numpy as np


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''

    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
import random
import numpy as np


seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()


for epoch_i in range(0, epochs):
    


    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    
    t0 = time.time()

    total_train_loss = 0

    
    model.train()

    for step, batch in enumerate(train_dataloader):


        if step % 40 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            

            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

     
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        
        model.zero_grad()        

        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        total_train_loss += loss.item()


        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()


        scheduler.step()


    avg_train_loss = total_train_loss / len(train_dataloader)            
    

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0


    for batch in validation_dataloader:
        
   
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
    
        with torch.no_grad():        

            
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        


    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


  Batch    40  of    578.    Elapsed: 0:00:52.
  Batch    80  of    578.    Elapsed: 0:01:43.
  Batch   120  of    578.    Elapsed: 0:02:34.
  Batch   160  of    578.    Elapsed: 0:03:25.
  Batch   200  of    578.    Elapsed: 0:04:16.
  Batch   240  of    578.    Elapsed: 0:05:07.
  Batch   280  of    578.    Elapsed: 0:05:58.
  Batch   320  of    578.    Elapsed: 0:06:49.
  Batch   360  of    578.    Elapsed: 0:07:40.
  Batch   400  of    578.    Elapsed: 0:08:31.
  Batch   440  of    578.    Elapsed: 0:09:22.
  Batch   480  of    578.    Elapsed: 0:10:13.
  Batch   520  of    578.    Elapsed: 0:11:04.
  Batch   560  of    578.    Elapsed: 0:11:55.

  Average training loss: 0.22
  Training epcoh took: 0:12:18

Running Validation...
  Accuracy: 0.90
  Validation Loss: 0.23
  Validation took: 0:00:25

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    578.    Elapsed: 0:00:51.
  Batch    80  of    578.    Elapsed: 0:01:42.
  Batch   120  of    578.    Elapsed: 0:02:33.
  Bat

In [25]:
import pandas as pd


pd.set_option('precision', 2)


df_stats = pd.DataFrame(data=training_stats)


df_stats = df_stats.set_index('epoch')


df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.22,0.23,0.90,0:12:18,0:00:25
2,0.14,0.16,0.94,0:12:17,0:00:25
3,0.09,0.22,0.94,0:12:15,0:00:25


In [0]:

sentences = X_test.values
labels = Y_test.values


input_ids = []
attention_masks = []


for sent in sentences:

    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 64,           
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
  
    input_ids.append(encoded_dict['input_ids'])
    

    attention_masks.append(encoded_dict['attention_mask'])


input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

 
batch_size = 32  


prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [27]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))


model.eval()

predictions , true_labels = [], []

for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  

  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]


  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  

  predictions.append(logits)
  true_labels.append(label_ids)

print('DONE.')

Predicting labels for 5,134 test sentences...
DONE.


In [0]:

flat_predictions = np.concatenate(predictions, axis=0)

flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

flat_true_labels = np.concatenate(true_labels, axis=0)

In [29]:
import sklearn
print (sklearn.metrics.classification_report(flat_predictions, flat_true_labels))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4561
           1       0.72      0.60      0.65       573

    accuracy                           0.93      5134
   macro avg       0.83      0.78      0.81      5134
weighted avg       0.92      0.93      0.93      5134



In [32]:
%ls
%cd "/content/drive/My Drive/Colab Notebooks/"

 2ndnew_train.tsv                                   task1.ipynb
 2ndnew_validation.tsv                              task2_en_training.tsv
 bert-embeddings-lstm.ipynb                         task2_en_validation.tsv
 bert-multiclass-classification.ipynb               task2_test_en.txt
 combined_brazil.json                               task2_train_en.txt
 combined_brazil_topics.json                       'task3 (1).ipynb'
 combined_india.json                                task3.ipynb
 combined_topics_brazil.json                        Template_Code.ipynb
 combined_usa.json                                  topic_combined_all_usa.json
 combined_usa_topics.json                           topic_combined_usa.json
'Copy of BBCTextClassifier-using-tokenizer.ipynb'   Untitled0.ipynb
'Copy of topic_modelling.ipynb'                     Untitled1.ipynb
 EN/                                                Untitled3.ipynb
 final_usa_data.json                                Untitled4.ipynb
 NLP_Task1_F

In [33]:

import os
output_dir = './model_save/'


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)


model_to_save = model.module if hasattr(model, 'module') else model  
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/vocab.json',
 './model_save/merges.txt',
 './model_save/special_tokens_map.json',
 './model_save/added_tokens.json')